In [2]:
import folium
import numpy as np
import pandas as pd
import scipy.stats as stats
import requests
import re
import matplotlib
%matplotlib inline
from bs4 import BeautifulSoup

In [7]:
topo_path = r'ch-cantons.topojson.json'
swiss_map = folium.Map(location=[46.8, 8.4], tiles='Stamen Terrain', zoom_start=7)
swiss_map.choropleth(geo_path=topo_path, topojson='objects.cantons')
swiss_map.save('swiss_map.html')
swiss_map

TODO:
- import data
- filter out 'Nicht zuteilbar - NA' and 'NPO'
- Groupby then request geonames


In [17]:
grants = pd.read_excel('P3_GrantExport.xlsx', header=0, usecols=["University","Approved Amount"])


In [34]:
grants.dropna(inplace=True)
grants

,University,Approved Amount
0,Nicht zuteilbar - NA,11619
1,Université de Genève - GE,41022
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732
3,Universität Basel - BS,52627
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042
5,Université de Fribourg - FR,53009
6,Université de Fribourg - FR,25403
7,Universität Zürich - ZH,47100
8,Université de Lausanne - LA,25814
9,Université de Genève - GE,360000


In [59]:
grants_filtered =grants.loc[(grants['University'] != "Nicht zuteilbar - NA") &
                            (grants['University'] != "NPO (Biblioth., Museen, Verwalt.) - NPO") &
                            (grants['Approved Amount'] != "data not included in P3")]

In [90]:
grants_grouped = grants_filtered.groupby('University').sum()

In [69]:
def request_GeoNames(name):
    url = 'http://api.geonames.org/searchJSON'
    payload = {
        'q':name,
        'country':'CH',
        'username':'bojbilla'
    }
    return requests.get(url,payload)

In [109]:
def split_and_request(name):
    r1 = request_GeoNames(name)
    if(r1.json()['totalResultsCount'] == 0):
        spt = name.split(' - ')
        for x in spt:
            r2 = request_GeoNames(x.strip())
            if(r2.json()['totalResultsCount']):
                return r2.json()
        return None
    else:
        r1.json()

In [110]:
jLists = split_and_request('EPF Lausanne - EsdPFL')
jLists is None

True

In [112]:
acc = 0
for g in grants_grouped.index:
    if(split_and_request(g) is None):
        print(g)
        acc = acc + 1
acc

AO Research Institute - AORI
Allergie- und Asthmaforschung - SIAF
Berner Fachhochschule - BFH
Biotechnologie Institut Thurgau - BITG
Centre de rech. sur l'environnement alpin - CREALP
Eidg. Hochschulinstitut für Berufsbildung - EHB
Ente Ospedaliero Cantonale - EOC
Fachhochschule Kalaidos - FHKD
Fachhochschule Ostschweiz - FHO
Facoltà di Teologia di Lugano - FTL
Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS
Firmen/Privatwirtschaft - FP
Forschungsanstalten Agroscope - AGS
Forschungsinstitut für Opthalmologie - IRO
Forschungsinstitut für biologischen Landbau - FIBL
Forschungskommission SAGW
Forschungskommission SAGW
Franklin University Switzerland - FUS
Friedrich Miescher Institute - FMI
HES de Suisse occidentale - HES-SO
Haute école pédagogique BE, JU, NE - HEPBEJUNE
Haute école pédagogique du canton de Vaud - HEPL
Haute école pédagogique fribourgeoise - HEPFR
Hochschule Luzern - HSLU
Idiap Research Institute - IDIAP
Inst. Suisse de Spéléologie et Karstologie - ISSKA
Inst. de Hautes

60